<img src="https://www.nasa.gov/wp-content/uploads/2023/03/704419main_meteor-gif-full_full.gif?w=1041" alt="Bay Area Fireball"></img>

# NASA API: World Map of Fireball Impacts

---

## Overview
This notebook will cover all the steps to access bright meteor and fireball impact data collected by NASA's CNEOS and produce a global plot of impact sites

1. Prerequisites
2. CNEOS Overview
3. API Overview and Setup
4. Imports
5. Access and Organize Data
6. Plot Earth Fireball Impacts
7. Summary

## Prerequisites
| Concepts | Importance | Notes |
| --- | --- | --- |
| [Intro to Matplotlib](https://foundations.projectpythia.org/core/matplotlib/matplotlib-basics.html) | Helfpul | |
| [Intro to Pandas](https://foundations.projectpythia.org/core/pandas/pandas.html) | Helpful | Familiarity with working with dataframes |

---

## CNEOS Overivew

A shooting star is a common term for a meteor and form bright trails of light that are often bright enough to be visible during the day. A "fireball" is a term for expectionally bright meteor that enters the Earth's atmosphere at high speeds. A meteor that form fireballs can be over one meter long. A "bolide" typically refers to a fireball that breaks up in the atmosphere. These objects are tracked by the [Center for Near Earth Object Studies (CNEOS)](https://cneos.jpl.nasa.gov/) and information about the Peak Brightness, Velocity, and Joules of energy radiatated can be retrieved from CNEOS via an API

---

## API Overview and Setup

There are many ways to gather data. Science and research entities like NASA are constantly producing and collecting data. As a result, attempting to collect and display live data can be difficult since new data is always being added. An API (Application Program Interface) is a live service that can be quered at the time that code is run to collect all the most current data. JPL's Center for Near-Earth Objects Studies [API "Fireball"](https://ssd-api.jpl.nasa.gov/doc/fireball.html) is an API that will return machine-readable data in the form of a JSON. This notebook will query the Fireball API for data from the last year of observations. The "last year" will depend on the year when this notebook is run, so the data is always up to date and relevant

### Setup API Access Key
Data is requested to the Fireball API via an API key. An API key is a string of numbers and letters that are unique to a specific user. API keys are required to query from an API and often are associated with an hourly limit on how often an API can be query. For NASA API's like Fireball there is an hourly limit of 1000 requests.

Sign up for an API key to have a unquie key generated on the [NASA API website](https://api.nasa.gov/). For the purposes of this notebook, the API key will be `DEMO_KEY`. `DEMO_KEY` is a valid API key used for lessons and documentation, but has a lower hourly limit of 30 requests per IP address per hour and a total of 50 requests per day. For the purposes of this notebook, replace `DEMO_KEY` with the API key that was generated for you on the [NASA API website](https://api.nasa.gov)

### Fireball Data API
The [Fireball Data API](https://ssd-api.jpl.nasa.gov/doc/fireball.html) returns data about Fireball atmospheric impact data reported by US Government sensors

All queries to the Fireball Data API make requests to `https://ssd-api.jpl.nasa.gov/fireball.api`

The API accepts different query parameters to filter the data

Directly querying `https://ssd-api.jpl.nasa.gov/fireball.api` will return all the data of fireball impacts in reverse-chronological order

| Parameter | Type | Description |
| --- | --- | --- |
| date-min | string | exclude data earlier than this date `YYYY-MM-DD` or date/time `YYYY-MM-DDThh:mm:ss` |
| date-max | string | exclude data later than this date `YYYY-MM-DD` or date/time `YYYY-MM-DDThh:mm:ss` |
| energy-min | string | exclude data with total-radiated-energy less than this positive value in joules ×1010 (e.g., `0.3` = 0.3×1010 joules) |
| energy-max | string | exclude data with total-radiated-energy greater than this (see energy-min) |
| impact-e-min | string | exclude data with estimated impact energy less than this positive value in kilotons (kt) (e.g., `0.08` kt) |
| impact-e-max | string | exclude data with total-radiated-energy greater than this (see impact-e-min) |
| alt-min | number | exclude data from objects with an altitude less than this (e.g., `22` meaning objects smaller than this) |
| alt-max | number | exclude data from objects with an altitude greater than this (e.g., `17.75` meaning objects larger than this) |
| req-loc | boolean | location (latitude and longitude) required; when set `true`, exclude data without a location |
| req-alt | boolean | altitude required; when set `true`, exclude data without an altitude |
| req-vel-comp | boolean | pre-entry velocity components required; when set `true`, exclude data without pre-entry velocity components |
| vel-comp | boolean | include pre-entry velocity components |
| sort | string | sort data on the specified field: “date”, “energy”, “impact-e”, “vel”, or “alt” (default sort order is ascending: prepend “-“ for descending) |
| limit | number | limit data to the first N results (where N is the specified number and must be an integer value greater than zero) |

Results from API will be returned as JSON with a number of fields

| Field | Description |
| --- | --- |
| date | date/time of peak brightness (GMT) |
| lat | latitude at peak brightness (degrees) |
| lon | longitude at peak brightness (degrees) |
| lat-dir | latitude direction (“N” or “S”) |
| lon-dir | latitude direction (“E” or “W”) |
| alt | altitude above the geoid at peak brightness (km) |
| energy | approximate total radiated energy (10^10 joules) |
| impact-e | approximate total impact energy (kt) |
| vx | pre-entry velocity (Earth centered X component, km/s) |
| vy | pre-entry velocity (Earth centered Y component, km/s) | 
| vz | pre-entry velocity (Earth centered Z component, km/s) |

## Example JSON result

Return the last 3 records `https://ssd-api.jpl.nasa.gov/fireball.api?limit=3`

```
{
    "signature":
        {
            "version":"1.0",
            "source":"NASA/JPL Fireball Data API"
        },
    "count":"3",
    "fields":["date","energy","impact-e","lat","lat-dir","lon","lon-dir","alt","vel"],
    "data":[
        ["2024-06-03 01:13:51","2.6","0.092","63.1","S","53.2","W","60.0",null],
        ["2024-06-01 23:24:59","7.9","0.25","1.0","S","15.9","W","26.5","12.6"],
        ["2024-05-27 03:19:36","8.4","0.26","1.7","N","39.4","W","56.0","39.2"]
    ]
}
```

## Imports

In [33]:
import requests                 # submit API request query
import pandas as pd             # organizes data retrieved by the API
import matplotlib.pyplot as plt # plot data on a world map
import io

## Request Fireball Data via API

Description

In [17]:
data_since = "2024-01-01"
last_year_fireball = requests.get(f"https://ssd-api.jpl.nasa.gov/fireball.api?date-min={data_since}")
print(last_year_fireball.status_code) # 200 is a valid request

200


In [45]:
# print out data
json = last_year_fireball.json()
print(f"fields = {json['fields']}")
print(f"data = {json['data']}")

fields = ['date', 'energy', 'impact-e', 'lat', 'lat-dir', 'lon', 'lon-dir', 'alt', 'vel']
data = [['2024-06-03 01:13:51', '2.6', '0.092', '63.1', 'S', '53.2', 'W', '60.0', None], ['2024-06-01 23:24:59', '7.9', '0.25', '1.0', 'S', '15.9', 'W', '26.5', '12.6'], ['2024-05-27 03:19:36', '8.4', '0.26', '1.7', 'N', '39.4', 'W', '56.0', '39.2'], ['2024-05-24 16:19:59', '5.4', '0.18', '6.6', 'N', '114.2', 'E', '42.0', '36.5'], ['2024-05-18 22:46:50', '3.7', '0.13', '41.0', 'N', '8.8', 'W', '74.3', '40.4'], ['2024-05-15 12:48:28', '2.8', '0.098', '49.8', 'N', '54.1', 'W', '64.2', None], ['2024-05-05 20:11:03', '3.7', '0.13', '7.1', 'S', '112.7', 'W', '39.0', '13.9'], ['2024-04-09 04:47:44', '3.3', '0.11', '36.5', 'N', '37.8', 'W', '36.3', None], ['2024-03-17 06:17:58', '3.7', '0.13', '27.6', 'N', '117.2', 'E', '42.1', '25.6'], ['2024-02-26 11:21:01', '3.7', '0.13', '22.9', 'N', '57.1', 'E', '33.1', '31.3'], ['2024-02-05 00:32:54', '4.6', '0.15', '13.8', 'S', '123.7', 'E', '23.0', '14.4']]


In [35]:
# convert JSON data to a Pandas dataframe

### A content subsection
Divide and conquer your objectives with Markdown subsections, which will populate the helpful navbar in Jupyter Lab and here on the Jupyter Book!

In [ ]:
# some subsection code
new = "helpful information"

### Another content subsection
Keep up the good work! A note, *try to avoid using code comments as narrative*, and instead let them only exist as brief clarifications where necessary.

## Your second content section
Here we can move on to our second objective, and we can demonstrate

### Subsection to the second section

#### a quick demonstration

##### of further and further

###### header levels

as well $m = a * t / h$ text! Similarly, you have access to other $\LaTeX$ equation [**functionality**](https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Typesetting%20Equations.html) via MathJax (demo below from link),

\begin{align}
\dot{x} & = \sigma(y-x) \\
\dot{y} & = \rho x - y - xz \\
\dot{z} & = -\beta z + xy
\end{align}

Check out [**any number of helpful Markdown resources**](https://www.markdownguide.org/basic-syntax/) for further customizing your notebooks and the [**Jupyter docs**](https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Working%20With%20Markdown%20Cells.html) for Jupyter-specific formatting information. Don't hesitate to ask questions if you have problems getting it to look *just right*.

## Last Section

If you're comfortable, and as we briefly used for our embedded logo up top, you can embed raw html into Jupyter Markdown cells (edit to see):

<div class="admonition alert alert-info">
    <p class="admonition-title" style="font-weight:bold">Info</p>
    Your relevant information here!
</div>

Feel free to copy this around and edit or play around with yourself. Some other `admonitions` you can put in:

<div class="admonition alert alert-success">
    <p class="admonition-title" style="font-weight:bold">Success</p>
    We got this done after all!
</div>

<div class="admonition alert alert-warning">
    <p class="admonition-title" style="font-weight:bold">Warning</p>
    Be careful!
</div>

<div class="admonition alert alert-danger">
    <p class="admonition-title" style="font-weight:bold">Danger</p>
    Scary stuff be here.
</div>

We also suggest checking out Jupyter Book's [brief demonstration](https://jupyterbook.org/content/metadata.html#jupyter-cell-tags) on adding cell tags to your cells in Jupyter Notebook, Lab, or manually. Using these cell tags can allow you to [customize](https://jupyterbook.org/interactive/hiding.html) how your code content is displayed and even [demonstrate errors](https://jupyterbook.org/content/execute.html#dealing-with-code-that-raises-errors) without altogether crashing our loyal army of machines!

---

## Summary
Add one final `---` marking the end of your body of content, and then conclude with a brief single paragraph summarizing at a high level the key pieces that were learned and how they tied to your objectives. Look to reiterate what the most important takeaways were.

### What's next?
Let Jupyter book tie this to the next (sequential) piece of content that people could move on to down below and in the sidebar. However, if this page uniquely enables your reader to tackle other nonsequential concepts throughout this book, or even external content, link to it here!

## Resources and references
Finally, be rigorous in your citations and references as necessary. Give credit where credit is due. Also, feel free to link to relevant external material, further reading, documentation, etc. Then you're done! Give yourself a quick review, a high five, and send us a pull request. A few final notes:
 - `Kernel > Restart Kernel and Run All Cells...` to confirm that your notebook will cleanly run from start to finish
 - `Kernel > Restart Kernel and Clear All Outputs...` before committing your notebook, our machines will do the heavy lifting
 - Take credit! Provide author contact information if you'd like; if so, consider adding information here at the bottom of your notebook
 - Give credit! Attribute appropriate authorship for referenced code, information, images, etc.
 - Only include what you're legally allowed: **no copyright infringement or plagiarism**
 
Thank you for your contribution!